In [1]:
!pip install feedparser

     |████████████████████████████████| 81kB 4.8MB/s 
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-cp37-none-any.whl size=6067 sha256=1fc7f0073673caef3b73ea2f1efc19d754d618f2626913b206ea7649d615b337
  Stored in directory: /root/.cache/pip/wheels/f1/80/5a/444ba08a550cdd241bd9baf8bae44be750efe370adb944506a
Successfully built sgmllib3k


In [3]:
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
import feedparser
import os
import numpy as np
from requests import Session
from multiprocessing.dummy import Pool as ThreadPool

In [4]:
s = Session()
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '\
                         'AppleWebKit/537.36 (KHTML, like Gecko) '\
                         'Chrome/75.0.3770.80 Safari/537.36'}
# Add headers
s.headers.update(headers)

In [159]:
def get_article_items(category,pages):
  pagesToGet = pages
  data = pd.DataFrame([])
  for page in range(1,pagesToGet+1):
      url = category + '/page/' + str(page)
      #an exception might be thrown, so the code should be in a try-except block
      try:
          #use the browser to get the url. This is suspicious command that might blow up.
          page=s.get(url)                             # this might throw an exception if something goes wrong.
      except Exception as e:                                   # this describes what to do if an exception is thrown
          error_type, error_obj, error_info = sys.exc_info()      # get the exception information
          print ('ERROR FOR LINK:',url)                          #print the link that cause the problem
          print (error_type, 'Line:', error_info.tb_lineno)     #print error info and line that threw the exception
          continue                                              #ignore this page. Abandon this and go back. 
      soup=BeautifulSoup(page.text,'lxml')
      links = soup.find_all('li',attrs={"class":"lx-stream__post-container"})
      for i in links:
          try: 
              title = i.find("span",attrs={'class':'lx-stream-post__header-text gs-u-align-middle'}).text.strip()
              link = "https://www.bbc.com"
              link += i.find('a',attrs={"class":"qa-heading-link lx-stream-post__header-link"})["href"]
              date = i.find('div',attrs={"class":"gs-u-display-flex"}).find("span",attrs={"class":"qa-post-auto-meta"}).text
              data = data.append(pd.DataFrame({'title': title, 'url': link,'date' : date}, index=[0]), ignore_index=True)
          except:
              continue
  return data
    

    

In [160]:
data = pd.DataFrame([])
list1 = []

In [161]:
%%time
vietnam = get_article_items("https://www.bbc.com/vietnamese/topics/ckdxnx1x5rnt",100)
vietnam["category"] = "vietnam"
list1.append(vietnam)

CPU times: user 10.2 s, sys: 250 ms, total: 10.4 s
Wall time: 1min 33s


In [162]:
vietnam.head()

(998, 4)

In [163]:
%%time
world = get_article_items("https://www.bbc.com/vietnamese/topics/cnlv9j1ekq0t",100)
world["category"] = "world"
list1.append(world)

CPU times: user 10.2 s, sys: 279 ms, total: 10.5 s
Wall time: 2min 13s


In [164]:
world.head()

,title,url,date,category
0,Putin cảnh báo phương Tây đừng vượt 'lằn ranh đỏ',https://www.bbc.com/vietnamese/world-56836367,15:43,world
1,Vụ George Floyd: 'Cuối cùng chúng tôi đã có th...,https://www.bbc.com/vietnamese/media-56833782,13:29,world
2,Huawei bác bỏ cáo buộc nghe lén ở Hà Lan,https://www.bbc.com/vietnamese/business-56815722,10:18 20 tháng 4 2021,world
3,Hội nghị thượng đỉnh ASEAN có thể chấm dứt đổ ...,https://www.bbc.com/vietnamese/world-56772917,5:39 20 tháng 4 2021,world
4,Cuba hiện nay là quá khứ của Việt Nam,https://www.bbc.com/vietnamese/forum-56800434,15:49 19 tháng 4 2021,world


In [165]:
%%time
economy = get_article_items("https://www.bbc.com/vietnamese/topics/cez1ey7nzj3t",20)
economy["category"] = "economy"
list1.append(economy)

CPU times: user 2.08 s, sys: 58.9 ms, total: 2.14 s
Wall time: 24.8 s


In [166]:
economy.head()

,title,url,date,category
0,Huawei bác bỏ cáo buộc nghe lén ở Hà Lan,https://www.bbc.com/vietnamese/business-56815722,10:18 20 tháng 4 2021,economy
1,Cựu TT Nguyễn Xuân Phúc không liên quan việc x...,https://www.bbc.com/vietnamese/vietnam-56797477,7:35 20 tháng 4 2021,economy
2,"Kinh tế Trung Quốc tăng trưởng 18,3% sau Covid",https://www.bbc.com/vietnamese/world-56755268,4:46 16 tháng 4 2021,economy
3,Alibaba bị siết vì 'lớn nhanh' hơn Đảng Cộng s...,https://www.bbc.com/vietnamese/business-56761377,12:29 15 tháng 4 2021,economy
4,Việt Nam: Kinh tế có đà tăng tốt kể cả sau dịc...,https://www.bbc.com/vietnamese/business-56750353,14:55 14 tháng 4 2021,economy


In [167]:
%%time
sports = get_article_items("https://www.bbc.com/vietnamese/topics/ckdxnx1k7zxt",7)
sports["category"] = "sports"
list1.append(sports)

CPU times: user 615 ms, sys: 11.1 ms, total: 626 ms
Wall time: 6.25 s


In [168]:
sports.head()

,title,url,date,category
0,European Super League làm dậy sóng làng bóng đá,https://www.bbc.com/vietnamese/sport-56814538,9:20 20 tháng 4 2021,sports
1,Bóng đá Việt Nam: Cầu thủ sướng - khổ ra sao?,https://www.bbc.com/vietnamese/vietnam-56772617,13:09 16 tháng 4 2021,sports
2,V-League: Sự đối lập giữa Hà Nội và HAGL,https://www.bbc.com/vietnamese/sport-56701343,8:53 10 tháng 4 2021,sports
3,Bóng đá Việt Nam: Thấy gì từ chấn thương của H...,https://www.bbc.com/vietnamese/vietnam-56509698,11:17 24 tháng 3 2021,sports
4,Novak Djokovic vô địch Australian Open,https://www.bbc.com/vietnamese/sport-56146261,11:19 21 tháng 2 2021,sports


In [169]:
df = vietnam.append(list1,ignore_index=True)

In [175]:
df = df.drop_duplicates(subset='url')

In [176]:
df.shape

(1752, 5)

In [171]:
s = Session()
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '\
                         'AppleWebKit/537.36 (KHTML, like Gecko) '\
                         'Chrome/75.0.3770.80 Safari/537.36'}
# Add headers
s.headers.update(headers)

    
def get_user_need(page):
    try:
        article = s.get(page)
        soup = BeautifulSoup(article.content, 'lxml')
        x = [i for i in soup.find("script",text = re.compile(r'(?<="campaignName".").+?(?=\")'))][0]
        return re.search(r'(?<="campaignName".").+?(?=\")',x)[0]
    except:
        return np.nan
    
def get_text(url):
    try:
        article = s.get(url)
        soup = BeautifulSoup(article.content, 'lxml')
        body = soup.find(role='main')
        text = [p.text for p in body.find_all('p',dir="ltr")]
        return ''.join(text)
    except:
        return np.nan


In [173]:
%%time
pool = ThreadPool(5)
user_need = pool.map(get_user_need, df["url"])
df["User Need"] = user_need

CPU times: user 5min 41s, sys: 1min 30s, total: 7min 11s
Wall time: 6min 47s


In [177]:
df["User Need"] = df["User Need"].apply(lambda s: re.sub(r"(WS - )", "", s) if pd.notnull(s) else s)


In [178]:
%%time
page_text = pool.map(get_text, df["url"])
df["text"] = page_text

CPU times: user 1min 35s, sys: 28.6 s, total: 2min 4s
Wall time: 1min 36s


In [180]:
df["User Need"].value_counts()

Update me              779
Give me perspective    617
Keep me on trend       140
Divert me               70
Educate me              69
Inspire me              48
CS                      20
News - Disclosure        1
Name: User Need, dtype: int64

In [181]:
df.to_csv("bbc_vietnam.csv",index=False)